In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Bidirectional,GlobalAveragePooling1D, Activation
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import tensorflow as tf
import cv2
import numpy as np
import os
import glob
import json
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [ ]:
# 1. Các biến cấu hình
DATA_PATH        = 'Data'                # thư mục gốc chứa các folder action
LABEL_MAP_PATH   = 'Logs/label_map.json'
BATCH_SIZE       = 32
AUTOTUNE         = tf.data.AUTOTUNE
VAL_SPLIT        = 0.1
TEST_SPLIT       = 0.1
# 2. Load label_map từ JSON
with open(LABEL_MAP_PATH, 'r', encoding='utf-8') as f:
    label_map = json.load(f)            # ví dụ: {"địa chỉ": 0, "miến điện": 1, ...}

# 3. Tạo danh sách tất cả các file .npz
#    Data structure: Data/<action_name>/*.npz
file_pattern = os.path.join(DATA_PATH, '**', '*.npz')
all_files = glob.glob(file_pattern, recursive=True)
print(f"Found {len(all_files)} samples.")

train_files, temp_files = train_test_split(
    all_files,
    test_size=VAL_SPLIT + TEST_SPLIT,  # ví dụ: 0.2 + 0.1 = 0.3
    shuffle=True,
    random_state=42,
    stratify=[os.path.basename(p).split('.')[0] for p in all_files]
)

val_files, test_files = train_test_split(
    temp_files,
    test_size=TEST_SPLIT / (VAL_SPLIT + TEST_SPLIT),  # ví dụ: 0.1 / 0.3 = 1/3
    shuffle=True,
    random_state=42,
    stratify=[os.path.basename(p).split('.')[0] for p in temp_files]
)

print(f"  Train samples: {len(train_files)}")
print(f"    Val samples: {len(val_files)}")
print(f"   Test samples: {len(test_files)}")

# 4. Hàm parse mỗi file .npz
def _load_npz(path):
    # path: scalar tf.string tensor
    npz_path = path.decode('utf-8')
    data = np.load(npz_path)
    seq   = data['sequence'].astype(np.float32)   # (60,126)
    lbl   = np.int32(data['label'])
    return seq, lbl

def parse_fn(path):
    seq, lbl = tf.numpy_function(
        func=_load_npz,
        inp=[path],
        Tout=[tf.float32, tf.int32]
    )
    # set shape để TF biết kích thước cố định
    seq.set_shape([60, 201])
    lbl.set_shape([])
    return seq, lbl
def make_dataset(file_list, shuffle=False, repeat=False):
    ds = tf.data.Dataset.from_tensor_slices(file_list)
    if shuffle:
        ds = ds.shuffle(len(file_list), reshuffle_each_iteration=True)
    if repeat:
        ds = ds.repeat()
    ds = ds.map(parse_fn, num_parallel_calls=AUTOTUNE)
    ds = ds.batch(BATCH_SIZE, drop_remainder=True)
    ds = ds.prefetch(AUTOTUNE)
    return ds

# 6. Tạo train_ds & val_ds
train_ds = make_dataset(train_files, shuffle=True, repeat=True)
val_ds   = make_dataset(val_files, shuffle=False, repeat=False)
test_ds  = make_dataset(test_files, shuffle=False, repeat=False)

# 7. Compute steps
steps_per_epoch = len(train_files) // BATCH_SIZE
validation_steps = len(val_files) // BATCH_SIZE

In [ ]:
inputs = tf.keras.Input(shape=(60, 201))

# Khối LSTM thứ nhất
x = Bidirectional(LSTM(256, return_sequences=True, dropout=0.3))(inputs)
x = BatchNormalization()(x)

# Khối LSTM thứ hai
x = Bidirectional(LSTM(256, return_sequences=True, dropout=0.3))(x)
x = BatchNormalization()(x)

# Khối LSTM thứ ba
x = Bidirectional(LSTM(256, dropout=0.3))(x)
x = BatchNormalization()(x)

# Các lớp Dense
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)

x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)

# Lớp đầu ra
outputs = Dense(2764, activation='softmax')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)
# Biên dịch mô hình
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# 1. Tạo thư mục lưu checkpoint (nếu chưa có)
checkpoint_dir = 'Models/checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_path = os.path.join(checkpoint_dir, 'final_model.keras')

# 2. Khởi tạo callbacks
callbacks = [
    # Lưu mô hình với val_loss thấp nhất
    ModelCheckpoint(
        filepath=checkpoint_path,
        monitor='val_loss',
        save_best_only=True,
        save_weights_only=False,  # lưu cả kiến trúc + weights
        verbose=1
    ),
    # Dừng training nếu 5 epoch liên tiếp không cải thiện val_loss
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    )
]

In [ ]:
model.fit(
    train_ds,
    epochs=100,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_ds,
    validation_steps=validation_steps,
    callbacks = callbacks
)